In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

# constructor for decision tree node
class DecisionTreeNode:
    def __init__(self, data, labels, nmin):
        self.data = data
        self.labels = labels
        self.nmin = nmin
        self.split_feature = None
        self.split_value = None
        self.left_child = None
        self.right_child = None
        self.label = self.find_dominant_label(labels)

    # get the dominant label for the current node
    def find_dominant_label(self, labels):
        unique_labels, counts = np.unique(labels, return_counts=True)
        dominant_label = unique_labels[np.argmax(counts)]
        return dominant_label

    # split the node based on the best gain
    def split(self):
        num_features = self.data.shape[1]
        best_gain = 0

        for feature in range(num_features):
            # evaluates the Information Gain for each unique number
            # the one that results in the highest gain is selected
            # as the splitting threshold for that particular feature
            unique_values = np.unique(self.data[:, feature])
            for value in unique_values:
                left_indices = self.data[:, feature] <= value
                right_indices = ~left_indices

                # skip if the split does not meet the minimum size requirement
                if np.sum(left_indices) < self.nmin or np.sum(right_indices) < self.nmin:
                    continue

                gain = self.calculate_information_gain(left_indices, right_indices)
                if gain > best_gain:
                    best_gain = gain
                    self.split_feature = feature
                    self.split_value = value

        # leaf node check
        if best_gain == 0:
            return False

        # perform the split
        left_indices = self.data[:, self.split_feature] <= self.split_value
        right_indices = ~left_indices

        left_data, left_labels = self.data[left_indices], self.labels[left_indices]
        right_data, right_labels = self.data[right_indices], self.labels[right_indices]

        self.left_child = DecisionTreeNode(left_data, left_labels, self.nmin)
        self.right_child = DecisionTreeNode(right_data, right_labels, self.nmin)

        # node split
        return True

    # IG = E(parent) - [weighted average] \cdot E(children)
    def calculate_information_gain(self, left_indices, right_indices):

        left_entropy = self.calculate_entropy(self.labels[left_indices])
        right_entropy = self.calculate_entropy(self.labels[right_indices])
        parent_entropy = self.calculate_entropy(self.labels)

        left_weight = np.sum(left_indices) / len(self.labels)
        right_weight = np.sum(right_indices) / len(self.labels)

        information_gain = parent_entropy - (left_weight * left_entropy + right_weight * right_entropy)
        return information_gain

    # E = - \sum p(X) \cdot log_{2}(p(X))
    # where p(X) = \frac{#x}{n}
    def calculate_entropy(self, labels):
        unique_labels, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))  # add a small constant to avoid log(0)
        return entropy

def create_decision_tree(data, labels, nmin):
    # initialize a decision tree
    # w/ features, targets, nmin
    root = DecisionTreeNode(data, labels, nmin)
    stack = [root]

    while stack:
        current_node = stack.pop()
        # if the conditions for splitting are met,
        # append the left and right child nodes to the stack
        if current_node.split() and len(current_node.labels) >= 2 * nmin:
            stack.append(current_node.right_child)
            stack.append(current_node.left_child)

    return root

def predict(tree, sample):
    # check if leaf node
    if tree.split_feature is None:
        return tree.label

    # check if the feature value of the sample corresponding
    # to the feature used for the split in the current node
    # is less than or equal to the split value
    if sample[tree.split_feature] <= tree.split_value:
        return predict(tree.left_child, sample)
    else:
        return predict(tree.right_child, sample)

def make_predictions(tree, data):
    predictions = [predict(tree, sample) for sample in data]
    return np.array(predictions)

def calculate_accuracy(predictions, true_labels):
    correct_predictions = np.sum(predictions == true_labels)
    accuracy = correct_predictions / len(true_labels)
    return accuracy

## Iris dataset

In [ ]:
# load dataset
# code from extras doc
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
iris_df = iris_df.replace({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

# extract features (all columns except last)
data = iris_df.iloc[:, :-1].values
# extract target labels (last column)
labels = iris_df.iloc[:, -1].values

# define values of nmin
nmin_values = [5, 10, 15, 20]

# perform 10-fold cross-validation for each value of nmin
for nmin in nmin_values:

    print(f"\nResults for nmin={nmin}\n")
    print(f"{'Fold':<5}{'Accuracy':<15}")
    print("-" * 20)

    accuracies = []

    # initialize KFold with 10 folds
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(kf.split(data), start=1):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # train decision tree on the training set
        decision_tree = create_decision_tree(X_train, y_train, nmin)

        # make predictions on the testing set
        predicted_labels = make_predictions(decision_tree, X_test)

        # calculate accuracy on the testing set
        accuracy = calculate_accuracy(predicted_labels, y_test)
        accuracies.append(accuracy)

        print(f"{i:<5}{accuracy*100:.2f}%")

    # calculate and print the average accuracy and std dev
    print("-" * 20)
    # Calculate average accuracy and standard deviation
    average_accuracy = np.mean(accuracies)
    std_dev_accuracy = np.std(accuracies)

    # Print average accuracy and standard deviation
    print(f"Average accuracy: {average_accuracy*100:.2f}%")
    print(f"Standard deviation: {std_dev_accuracy*100:.2f}%")


: 

## Spambase dataset

In [2]:
# load dataset
# code from extras doc
spambase_df = pd.read_csv("spambase.csv")

# extract features (all columns except last)
data = spambase_df.iloc[:, :-1].values
# extract target labels (last column)
labels = spambase_df.iloc[:, -1].values

# define values of nmin
nmin_values = [5, 10, 15, 20, 25]

# perform 10-fold cross-validation for each value of nmin
for nmin in nmin_values:

    print(f"\nResults for nmin={nmin}\n")
    print(f"{'Fold':<5}{'Accuracy':<15}")
    print("-" * 20)

    accuracies = []

    # initialize KFold with 10 folds
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(kf.split(data), start=1):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        # train decision tree on the training set
        decision_tree = create_decision_tree(X_train, y_train, nmin)

        # make predictions on the testing set
        predicted_labels = make_predictions(decision_tree, X_test)

        # calculate accuracy on the testing set
        accuracy = calculate_accuracy(predicted_labels, y_test)
        accuracies.append(accuracy)

        print(f"{i:<5}{accuracy*100:.2f}%")

    # calculate and print the average accuracy and std dev
    print("-" * 20)
    # Calculate average accuracy and standard deviation
    average_accuracy = np.mean(accuracies)
    std_dev_accuracy = np.std(accuracies)

    # Print average accuracy and standard deviation
    print(f"Average accuracy: {average_accuracy*100:.2f}%")
    print(f"Standard deviation: {std_dev_accuracy*100:.2f}%")



Results for nmin=5

Fold Accuracy       
--------------------
1    91.96%
2    91.74%
3    91.52%
4    94.13%
5    91.96%
6    90.65%
7    91.52%
8    91.74%
9    93.48%
10   93.70%
--------------------
Average accuracy: 92.24%
Standard deviation: 1.07%

Results for nmin=10

Fold Accuracy       
--------------------
1    92.39%
2    91.30%
3    92.61%
4    93.91%
5    91.52%
6    90.00%
7    91.74%
8    92.17%
9    91.74%
10   93.91%
--------------------
Average accuracy: 92.13%
Standard deviation: 1.12%

Results for nmin=15

Fold Accuracy       
--------------------
1    91.74%
2    89.78%
3    90.22%
4    93.91%
5    91.09%
6    90.22%
7    90.43%
8    91.74%
9    91.52%
10   92.39%
--------------------
Average accuracy: 91.30%
Standard deviation: 1.18%

Results for nmin=20

Fold Accuracy       
--------------------
1    89.78%
2    89.57%
3    89.57%
4    93.70%
5    91.30%
6    90.87%
7    91.74%
8    92.39%
9    91.96%
10   92.83%
--------------------
Average accuracy: 91.37%
Sta